<a href="https://colab.research.google.com/github/ethandavenport/Optimization-I-Project-2/blob/main/Optimization_HW_pt_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 26.7 MB/s eta 0:00:00


In [2]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np

In [72]:
url2 = "https://raw.githubusercontent.com/ethandavenport/Optimization-I-Project-2/refs/heads/main/roi_company2.csv"
df_roi2 = pd.read_csv(url2)

# -----------------------------
# SETUP (amounts in MILLIONS)
# -----------------------------
budget = 10.0        # total budget in millions
platform_cap = 3.0   # per-platform max in millions
bigM = budget        # safe big-M for "inf" tiers

# Ensure Width exists (based on df_roi2)
def tier_width(row):
    if np.isfinite(row['UpperBound']):
        return max(0.0, row['UpperBound'] - row['LowerBound'])
    else:
        return bigM  # allow up to whole budget for "inf"
df2 = df_roi2.copy()
df2['Width'] = df2.apply(tier_width, axis=1)

# -----------------------------
# BUILD MODEL FROM df_roi2 ONLY
# -----------------------------
m = gp.Model()
m.Params.OutputFlag = 0

# Decision vars: amount invested in each tier (millions)
vars_by_index = {}
for idx, row in df2.iterrows():
    name = f"x_{int(idx)}_{row['Platform']}_t{int(row['Tier'])}"
    ub = float(row['Width'])
    # keep same pattern as your LP
    if ub <= 0:
        v = m.addVar(lb=0.0, ub=0.0, name=name)
    else:
        v = m.addVar(lb=0.0, ub=ub, name=name)
    vars_by_index[idx] = v

# Objective: maximize total return = sum(ROI * invested_in_tier)
obj_expr = gp.quicksum(df2.loc[idx, 'ROI'] * vars_by_index[idx] for idx in vars_by_index)
m.setObjective(obj_expr, GRB.MAXIMIZE)

# Helper: platform -> list of tier indices (built from df_roi2)
platform_to_indices = {}
for idx, row in df2.iterrows():
    platform_to_indices.setdefault(row['Platform'], []).append(idx)

# -----------------------------
# SAME BUSINESS CONSTRAINTS
# -----------------------------
# 1) Total budget
m.addConstr(gp.quicksum(vars_by_index[idx] for idx in vars_by_index) <= budget)

# 2) Per-platform cap
for platform, idx_list in platform_to_indices.items():
    m.addConstr(gp.quicksum(vars_by_index[idx] for idx in idx_list) <= platform_cap)

# (a) Print + TV <= Facebook + Email
def platform_total(p):
    idxs = platform_to_indices.get(p, [])
    return gp.quicksum(vars_by_index[i] for i in idxs)

m.addConstr(
    platform_total('Print') + platform_total('TV')
    <= platform_total('Facebook') + platform_total('Email')
)

# (b) Social >= 2 * (SEO + AdWords)
social_platforms = ['Facebook', 'LinkedIn', 'Instagram', 'Snapchat', 'Twitter']
social_sum = gp.quicksum(platform_total(p) for p in social_platforms)
m.addConstr(social_sum >= 2.0 * (platform_total('SEO') + platform_total('AdWords')))

# -----------------------------
# MIP ADD-ON: NO SKIPPING
# -----------------------------
# 1) Binary activation per tier
z_by_index = {}
for idx, row in df2.iterrows():
    z_by_index[idx] = m.addVar(vtype=GRB.BINARY,
                               name=f"z_{int(idx)}_{row['Platform']}_t{int(row['Tier'])}")

# 2) Gate: no spend unless tier is on
for idx, row in df2.iterrows():
    m.addConstr(vars_by_index[idx] <= row['Width'] * z_by_index[idx],
                name=f"gate_{int(idx)}_{row['Platform']}_t{int(row['Tier'])}")

# 3) Chain (no activation skipping): if higher tier is on, lower must be on  -> z_prev >= z_next
for platform, idx_list in platform_to_indices.items():
    idx_list_sorted = sorted(
        idx_list,
        key=lambda k: (float(df2.loc[k, 'LowerBound']), int(df2.loc[k, 'Tier']))
    )
    for prev_idx, next_idx in zip(idx_list_sorted[:-1], idx_list_sorted[1:]):
        m.addConstr(z_by_index[prev_idx] >= z_by_index[next_idx],
                    name=f"order_{platform}_t{int(df2.loc[prev_idx,'Tier'])}_to_t{int(df2.loc[next_idx,'Tier'])}")

# 4) Fill-up (no spend skipping): must fully fund tier j before any spend in tier j+1
for platform, idx_list in platform_to_indices.items():
    idx_list_sorted = sorted(
        idx_list,
        key=lambda k: (float(df2.loc[k, 'LowerBound']), int(df2.loc[k, 'Tier']))
    )
    for prev_idx, next_idx in zip(idx_list_sorted[:-1], idx_list_sorted[1:]):
        m.addConstr(
            vars_by_index[prev_idx] >= df2.loc[prev_idx, 'Width'] * z_by_index[next_idx],
            name=f"fill_{platform}_t{int(df2.loc[prev_idx,'Tier'])}_before_t{int(df2.loc[next_idx,'Tier'])}"
        )



# -----------------------------
# SOLVE + CLEAN REPORT
# -----------------------------
m.optimize()

EPS = 1e-9
print(f"Objective (total return): {m.objVal:,.6f}")
for platform, idx_list in platform_to_indices.items():
    invested = sum(vars_by_index[i].X for i in idx_list)
    tiers_on = sorted([int(df2.loc[i, 'Tier']) for i in idx_list if vars_by_index[i].X > EPS])
    print(f"{platform:12s}  invested={invested:,.3f}M   tiers_on={tiers_on}")


Objective (total return): 0.452827
Print         invested=3.000M   tiers_on=[1, 2]
TV            invested=0.000M   tiers_on=[]
SEO           invested=0.000M   tiers_on=[]
AdWords       invested=2.333M   tiers_on=[1, 2]
Facebook      invested=3.000M   tiers_on=[1]
LinkedIn      invested=1.667M   tiers_on=[1]
Instagram     invested=0.000M   tiers_on=[]
Snapchat      invested=0.000M   tiers_on=[]
Twitter       invested=0.000M   tiers_on=[]
Email         invested=0.000M   tiers_on=[]
